# Mock Lambda Function

In [1]:
test_comment = "The unquestioned value of crypto currencies is that they provide an easy way for organized crime syndicates of all stripes to launder their ill gotten millions and billions while making suckers out of governments and the rest of us."

test_event = {'comment': test_comment}
test_context = None

In [2]:
import boto3

s3 = boto3.resource('s3')
obj = s3.Object('sagemaker-us-east-1-228889150161','data/nyt-features/vocab.json')
obj.download_file('vocab.json')


In [23]:
import json 
import re 
from collections import Counter
import boto3
import io

def load_vocab():
    with open('vocab.json', 'r') as json_file:
        return json.load(json_file)

def mock_function(event, context):
    vocab = load_vocab()
    tokens = re.split(r'\W+', event['comment'])
    
    lowercase = map(lambda token: token.lower(), tokens)
    vocab_words = filter(lambda token: token in vocab, lowercase)
    idx = map(lambda token: vocab[token], vocab_words)
    counts = Counter(idx)
    
    #https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html
    keys = sorted(counts.keys())
    values = list(map(lambda key: counts[key], keys))
    features = { "keys": keys, "shape": [1000],"values": values }
    
    ntm_input = {"instances":[{"data": {"features": features}}]}
    req_body = json.dumps(ntm_input)
    print(req_body)
    
    client = boto3.client('sagemaker-runtime')
    resp = client.invoke_endpoint(
        EndpointName='ntm-nyt',
        Body=req_body.encode('utf-8'),
        ContentType='application/json',
        Accept='application/json'
    )
        
    predictions = json.load(resp["Body"])
    return predictions

In [24]:
response = mock_function(test_event, test_context)

{"instances": [{"data": {"features": {"keys": [15, 186, 275, 414, 416, 486, 579, 599, 849], "shape": [1000], "values": [1, 1, 1, 1, 1, 1, 1, 1, 1]}}}]}


In [25]:
print(response)

{'predictions': [{'topic_weights': [0.049695503, 0.0527563505, 0.0490425341, 0.0609627143, 0.0683583915, 0.0421626978, 0.0433707982, 0.058722306, 0.036552988, 0.0425718091, 0.048611436, 0.0549974181, 0.0513481162, 0.0382936932, 0.0424936749, 0.0486433618, 0.040738333, 0.0580059662, 0.0668672547, 0.045804657]}]}
